# Perform VADER Sentiment on Bitcoin and Litecoin to determine news stories sentiment

In [1]:
# Initial imports
import os
from pathlib import Path
import pandas as pd
from newsapi import NewsApiClient
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from dotenv import load_dotenv
load_dotenv()


True

In [2]:
# Download/Update the VADER Lexicon
nltk.download('vader_lexicon')

# Initialize the VADER sentiment analyzer
analyzer = SentimentIntensityAnalyzer()



[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/mrnagleJR97/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [3]:
# Reading the News API key enviroment variable
api_key = os.getenv("news_api")



In [4]:
# Create a newsapi client
newsapi = NewsApiClient(api_key=api_key)



In [5]:
# Fetch all the news about Facebook Libra
crypto_headlines = newsapi.get_everything(
    q="litecoin AND bitcoin",
    language="en",
    page_size=100,
    sort_by="relevancy"
)

# Print total articles
print(f"Total articles about Litecoin and Bitcoin: {crypto_headlines['totalResults']}")

# Show sample article
crypto_headlines["articles"][0]



Total articles about Litecoin and Bitcoin: 370


{'source': {'id': 'techcrunch', 'name': 'TechCrunch'},
 'author': 'Sarah Perez',
 'title': 'PayPal expands the ability to buy, hold and sell cryptocurrency to the U.K.',
 'description': 'PayPal will now allow users outside the U.S. to buy, hold and sell cryptocurrency for the first time. The company announced today the launch of a new service that will allow customers in the U.K. to select between four types of cryptocurrencies — including Bi…',
 'url': 'http://techcrunch.com/2021/08/23/paypal-expands-the-ability-to-buy-hold-and-sell-cryptocurrency-to-the-u-k/',
 'urlToImage': 'https://techcrunch.com/wp-content/uploads/2020/11/GettyImages-887657568.jpg?w=600',
 'publishedAt': '2021-08-23T13:49:45Z',
 'content': 'PayPal will now allow users outside the U.S. to buy, hold and sell cryptocurrency for the first time. The company announced today the launch of a new service that will allow customers in the U.K. to … [+4420 chars]'}

In [6]:
# Create the Facebook Libra sentiment scores DataFrame
crypto_sentiments = []

for article in crypto_headlines["articles"]:
    try:
        text = article["content"]
        date = article["publishedAt"][:10]
        sentiment = analyzer.polarity_scores(text)
        compound = sentiment["compound"]
        pos = sentiment["pos"]
        neu = sentiment["neu"]
        neg = sentiment["neg"]
        
        crypto_sentiments.append({
            "text": text,
            "date": date,
            "compound": compound,
            "positive": pos,
            "negative": neg,
            "neutral": neu
            
        })
        
    except AttributeError:
        pass
    
# Create DataFrame
crypto_df = pd.DataFrame(crypto_sentiments)

# Reorder DataFrame columns
cols = ["date", "text", "compound", "positive", "negative", "neutral"]
crypto_df = crypto_df[cols]

crypto_df.head()



,date,text,compound,positive,negative,neutral
0,2021-08-23,PayPal will now allow users outside the U.S. t...,0.4215,0.098,0.0,0.902
1,2021-08-23,"PayPal is bringing the ability to buy, hold an...",0.6908,0.161,0.0,0.839
2,2021-08-23,"PayPal is to allow users in the UK to buy, hol...",0.4215,0.095,0.0,0.905
3,2021-08-24,PayPal launched its crypto services in the UKi...,0.2263,0.094,0.0,0.906
4,2021-09-07,Andrew Kelly/Reuters\r\n<ul><li>Bitcoin and et...,0.5106,0.137,0.0,0.863


In [7]:
# Get descriptive stats from the DataFrame
crypto_df.describe()



,compound,positive,negative,neutral
count,100.000000,100.00000,100.000000,100.000000
mean,0.261394,0.07652,0.017210,0.906280
std,0.353131,0.07028,0.046797,0.075729
min,-0.893400,0.00000,0.000000,0.687000
25%,0.000000,0.00000,0.000000,0.861500
50%,0.340000,0.07200,0.000000,0.917500
75%,0.534600,0.11375,0.000000,0.961000
max,0.886000,0.31300,0.312000,1.000000
